In [ ]:
# ============================================================
#   1. INSTALL & IMPORT
# ============================================================
!pip install -q sentence-transformers scikit-learn

import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# ============================================================
#   2. LOAD DATASET
# ============================================================
filename = "daftar_objek_50.csv"
df = pd.read_csv(filename)

# Tambah kolom untuk menyimpan clue history (untuk training NB)
if "clue_history" not in df.columns:
    df["clue_history"] = df["deskripsi"].copy()

print(f"✅ Dataset loaded: {len(df)} objek")

# ============================================================
#   3. LOAD MODELS
# ============================================================
# Sentence Transformer (Semantic)
st_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# TF-IDF + Naive Bayes (NLP)
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
nb_model = MultinomialNB(alpha=0.1)
label_encoder = LabelEncoder()

# ============================================================
#   4. BUILD EMBEDDINGS & TRAIN NLP
# ============================================================
def gabung_teks(row):
    return f"{row['objek']} - {row['deskripsi']}"

def rebuild_all_models():
    global objek_list, objek_only, objek_embeddings
    global X_tfidf, y_encoded

    # === Sentence Transformer ===
    objek_list = df.apply(gabung_teks, axis=1).tolist()
    objek_only = df["objek"].tolist()
    objek_embeddings = st_model.encode(objek_list, convert_to_tensor=True)

    # === Naive Bayes ===
    # Buat training data dari clue_history
    train_texts = []
    train_labels = []

    for _, row in df.iterrows():
        clues = row["clue_history"].split(", ")
        for c in clues:
            if len(c.strip()) > 2:
                train_texts.append(c.strip().lower())
                train_labels.append(row["objek"])

    if len(train_texts) > 0:
        X_tfidf = tfidf.fit_transform(train_texts)
        y_encoded = label_encoder.fit_transform(train_labels)
        nb_model.fit(X_tfidf, y_encoded)

    print("✅ Sentence Transformer + Naive Bayes siap!")

rebuild_all_models()

# ============================================================
#   5. FUNGSI PREDIKSI HYBRID
# ============================================================
def predict_semantic(clue, top_k=10):
    """Prediksi pakai Sentence Transformer"""
    clue_emb = st_model.encode(clue, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(clue_emb, objek_embeddings)[0]

    hasil = {}
    for i, score in enumerate(scores):
        hasil[objek_only[i]] = float(score)
    return hasil

def predict_naive_bayes(clue):
    """Prediksi pakai Naive Bayes"""
    try:
        X_new = tfidf.transform([clue.lower()])
        proba = nb_model.predict_proba(X_new)[0]
        classes = label_encoder.classes_

        hasil = {}
        for i, p in enumerate(proba):
            hasil[classes[i]] = float(p)
        return hasil
    except:
        return {}

def predict_hybrid(clue, top_k=5, weight_st=0.5, weight_nb=0.5):
    """Gabungan Semantic + Naive Bayes"""

    # Dapat skor dari kedua model
    scores_st = predict_semantic(clue)
    scores_nb = predict_naive_bayes(clue)

    # Normalisasi skor ST ke range 0-1
    max_st = max(scores_st.values()) if scores_st else 1
    min_st = min(scores_st.values()) if scores_st else 0
    range_st = max_st - min_st if max_st != min_st else 1

    # Kombinasi skor
    final_scores = {}
    for objek in objek_only:
        st_norm = (scores_st.get(objek, 0) - min_st) / range_st
        nb_score = scores_nb.get(objek, 0)

        # Weighted average
        final_scores[objek] = (weight_st * st_norm) + (weight_nb * nb_score)

    # Sort dan ambil top_k
    sorted_scores = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)

    hasil = []
    for objek, score in sorted_scores[:top_k]:
        hasil.append({
            "objek": objek,
            "score": round(score, 4),
            "st": round(scores_st.get(objek, 0), 4),
            "nb": round(scores_nb.get(objek, 0), 4)
        })
    return hasil

# ============================================================
#   6. FUNGSI BELAJAR DARI FEEDBACK
# ============================================================
def learn_from_feedback(clue, jawaban_benar, deskripsi_tambahan=None):
    global df

    idx = df[df["objek"].str.lower() == jawaban_benar.lower()].index

    if len(idx) > 0:
        i = idx[0]
        # Update deskripsi
        old_desc = df.at[i, "deskripsi"]
        tambahan = deskripsi_tambahan if deskripsi_tambahan else clue
        df.at[i, "deskripsi"] = f"{old_desc}, {tambahan}"

        # Update clue_history (untuk Naive Bayes) - repeat 3x untuk boost
        old_clues = df.at[i, "clue_history"]
        boost = f"{clue}, {clue}, {clue}"  # Triple boost!
        df.at[i, "clue_history"] = f"{old_clues}, {boost}"

        print(f"🔄 '{jawaban_benar}' diperkuat dengan: {tambahan}")
    else:
        desc = deskripsi_tambahan if deskripsi_tambahan else f"{jawaban_benar} adalah {clue}"
        boost = f"{clue}, {clue}, {clue}"
        baru = pd.DataFrame({
            "objek": [jawaban_benar],
            "deskripsi": [desc],
            "clue_history": [f"{desc}, {boost}"]
        })
        df = pd.concat([df, baru], ignore_index=True)
        print(f"➕ Objek baru: {jawaban_benar}")

    # Rebuild KEDUA model
    rebuild_all_models()
    print("✅ Semantic + NLP model diperbarui!")

# ============================================================
#   7. AUTO-SAVE
# ============================================================
feedback_counter = 0
auto_save_every = 3
save_filename = "daftar_objek_updated.csv"

def save_dataset(filename=None):
    global save_filename
    if filename:
        save_filename = filename
    df.to_csv(save_filename, index=False)
    print(f"💾 Disimpan: {save_filename} ({len(df)} objek)")

def check_auto_save():
    global feedback_counter
    feedback_counter += 1
    print(f"📊 Feedback: {feedback_counter}/{auto_save_every}")
    if feedback_counter >= auto_save_every:
        save_dataset()
        feedback_counter = 0

# ============================================================
#   8. GAME INTERAKTIF
# ============================================================
def play_game():
    print("\n" + "="*55)
    print("🎮 GAME TEBAK OBJEK - ML + NLP EDITION")
    print("="*55)
    print("Perintah: 'quit' keluar | 'save' simpan | 'stat' statistik\n")

    stats = {"benar": 0, "salah": 0}

    while True:
        clue = input("🔍 Clue: ").strip()

        if clue.lower() == "quit":
            save_dataset()
            akurasi = stats["benar"]/(stats["benar"]+stats["salah"])*100 if (stats["benar"]+stats["salah"]) > 0 else 0
            print(f"\n📈 Akurasi sesi ini: {akurasi:.1f}% ({stats['benar']}/{stats['benar']+stats['salah']})")
            print("👋 Sampai jumpa!")
            break
        elif clue.lower() == "save":
            save_dataset()
            continue
        elif clue.lower() == "stat":
            print(f"📊 Benar: {stats['benar']} | Salah: {stats['salah']}")
            continue
        elif not clue:
            continue

        hasil = predict_hybrid(clue, top_k=5)

        print(f"\n🤖 Tebakan (ST=Semantic, NB=NaiveBayes):")
        for i, h in enumerate(hasil, 1):
            print(f"   {i}. {h['objek']:20} [Final: {h['score']:.3f} | ST: {h['st']:.3f} | NB: {h['nb']:.3f}]")

        tebakan = hasil[0]["objek"]
        print(f"\n🎯 Tebakan: {tebakan}")

        feedback = input("✅ Benar? (y/n/skip): ").strip().lower()

        if feedback == "y":
            print("🎉 Benar!")
            stats["benar"] += 1
            learn_from_feedback(clue, tebakan)
            check_auto_save()
        elif feedback == "n":
            stats["salah"] += 1
            jawaban = input("📝 Jawaban benar: ").strip()
            if jawaban:
                desc = input("📖 Deskripsi (enter skip): ").strip()
                learn_from_feedback(clue, jawaban, desc if desc else None)
                check_auto_save()
        else:
            print("⏭️ Skip")

        print("-"*55 + "\n")

# ============================================================
#   9. START
# ============================================================
print("\n🚀 Ketik play_game() untuk mulai!")
print("📊 Atau predict_hybrid('clue') untuk test\n")